# APSG tutorial - structural geology package for Python
[Ondrej Lexa](https://petrol.natur.cuni.cz/~ondro/)  
Institute of Petrology and Structural Geology  
Faculty of Science, Charles University  
[lexa.ondrej@gmail.com](mailto:lexa.ondrej@gmail.com)

**APSG** defines several new python classes to easily manage, analyze and visualize orientation structural geology data. Base class ``Vec3`` is designed to represents **vectorial data** and is derived from Numpy array. It offers several new method which will be explained in following examples.

In [ ]:
%pylab inline

## Basic usage

**APSG** module could be imported either into own name space or into active one for easier interactive work

In [ ]:
from apsg import *
plt.rcParams['figure.figsize'] = (10, 8)

### Basic operations with vectors
Instance of vector object ``Vec3`` could be created from any iterable object as list, tuple or array

In [ ]:
u = Vec3([1, -2, 3])
v = Vec3([-2, 1, 1])

For common vector operation we can use standard mathematical operators or special methods using dot notation

In [ ]:
u + v

In [ ]:
u - v

In [ ]:
3*u - 2*v

Its magnitude or length is most commonly defined as its Euclidean norm and could be calculated using ``abs``

In [ ]:
abs(v)

In [ ]:
abs(u + v)

For *dot product* we can use multiplication operator ``*`` or ``dot`` method

In [ ]:
u*v

In [ ]:
u.dot(v)

For *cross product* we can use operator ``**`` or method ``cross``

In [ ]:
u**v

In [ ]:
u.cross(v)

To project vector ``u`` onto vector ``v`` we can use method ``proj``

In [ ]:
u.proj(v)

To find angle (in degrees) between to vectors we use method ``angle``

In [ ]:
u.angle(v)

Method ``rotate`` provide possibility to rotate vector around another vector. For example, to rotate vector ``u`` around vector ``v`` for 45°

In [ ]:
u.rotate(v, 45)

We can check whether the rotated vector has same angle as original to axis of rotation

In [ ]:
v.angle(u), v.angle(u.rotate(v, 45))

## Classes Lin and Fol
To work with orientation data in structural geology, **APSG** provide two classes derived from `Vec3` class. There is `Fol` class to represent planar features (planes) and `Lin` class to represent linear feature (lines). Both classes provide all `Vec3` methods, but they differ in way how instance is created and how some operations evaluated, as both `Fol` and `Lin` are designed to represents **axial data**.

To create instance of ``Lin`` or ``Fol`` class, we have to provide dip direction and dip (for lines the plunge direction and plunge), both in degrees:  
*There is very limited support for other notations. However, it can change in future...*
<img src="images/dipdir.png">

In [ ]:
Lin(120, 60), Fol(216, 62)

or we can create instance from ``Vec3`` object using ``aslin`` and ``asfol`` properties  
*for planar feature vector represents it's normal vector*

In [ ]:
u.aslin, u.asfol

Features like `Vec3`, `Lin` and `Fol` could be easily visualized on Stereographic projection using `StereoNet` class. Providing any number of arguments results in immediate plot. More sophisticated usage will be described later.  
*Note semicolon character on the end of the line. It supress object representation output.*

In [ ]:
StereoNet(Lin(120, 60), Fol(216, 62), u, u.asfol);

### Vec3 methods for Lin and Fol


To find angle between two linear or planar features we can use method ``angle``

In [ ]:
l1 = Lin(110, 40)
l2 = Lin(160, 30)
l1.angle(l2)

In [ ]:
p1 = Fol(330, 50)
p2 = Fol(250, 40)
p1.angle(p2)

We can use *cross product* to construct planar feature defined by two linear features

In [ ]:
l1**l2

In [ ]:
StereoNet(l1, l2, l1**l2);

or to construct linear feature defined as intersection of two planar features

In [ ]:
p1**p2

In [ ]:
StereoNet(p1, p2, p1**p2);

*Cross product* of planar and linear features could be used to construct plane defined by linear feature and normal of planar feature

In [ ]:
l2**p2

In [ ]:
StereoNet(l2, p2, l2**p2);

or to find perpendicular linear feature on given plane

In [ ]:
p2**l2

In [ ]:
StereoNet(p2, l2, p2**l2);

In [ ]:
l2.angle(p2**l2)

To rotate structural features we can use method ``rotate``

In [ ]:
p2.rotate(p1**p2, 45)

In [ ]:
StereoNet(p2, p1, p2.rotate(p1**p2, 45));

## Classes Pair and Fault
To work with paired orientation data like foliations and lineations or fault data in structural geology, **APSG** provide two base ``Pair`` class and derived ``Fault`` class. Both classes are instantiated providing dip direction and dip of planar and linear measurements, which are automatically orthogonalized. If misfit is too high, warning is raised. The ``Fault`` class expects one more argument providing sense of movement information, either 1 or -1.

To create instance of ``Pair`` class, we have to provide dip direction and dip of planar and linear feature, both in degrees

In [ ]:
p = Pair(120, 40, 162, 28)
p

In [ ]:
p.misfit

Planar and linear features are accessible using ``fol`` and ``lin`` properties

In [ ]:
p.fol, p.lin

In [ ]:
StereoNet(p);

To rotate ``Pair`` instance we can use ``rotate`` method

In [ ]:
p.rotate(Lin(45, 10), 60)

Instantiation of ``Fault`` class is similar, we just have to provide argument to define sense of movement

In [ ]:
f = Fault(120, 60, 110, 58, -1)  # -1 for normal fault
f

In [ ]:
StereoNet(f);

Note the change in sense of movement after ``Fault`` rotation

In [ ]:
f.rotate(Lin(45, 10), 60)

For simple fault analyses ``Fault`` class also provide ``p``, ``t``, ``m`` and ``d`` properties to get PT-axes, kinematic plane and dihedra separation plane

In [ ]:
f.p, f.t, f.m, f.d

In [ ]:
StereoNet(f, f.p, f.t, f.m, f.d);

## Group class
``Group`` class serve as a homogeneous container for ``Lin``, ``Fol`` and ``Vec3`` objects. It allows grouping of features either for visualization or batch analysis

In [ ]:
g = Group([Lin(120,60), Lin(116,50), Lin(132,45), Lin(90,60), Lin(84,52)], name='L1')
g

To simplify interactive group creation, you can use function ``G``

In [ ]:
g = G('120 60 116 50 132 45 90 60 84 52', name='L1')
g

Method ``len`` returns number of features in group

In [ ]:
len(g)

In [ ]:
StereoNet(g);

Most of the ``Lin``, ``Fol`` and ``Vec3`` methods could be used for ``Group`` as well. For example, to measure angles between all features in group and another feature, we can use method ``angle``

In [ ]:
g.angle(Lin(110,50))

To rotate all features in group around another feature, we can use method ``rotate``

In [ ]:
gr = g.rotate(Lin(270, 60), 60)

In [ ]:
StereoNet(g, gr);

To show data in list you can use ``data`` property

In [ ]:
gr.data

Property ``R`` gives mean or resultant of all features in group. Note that ``Lin`` and ``Fol`` are axial in nature, so resultant vector is not reliable. You can use ``ortensor`` property.

In [ ]:
g.R

In [ ]:
StereoNet(g, g.R);

``Group`` class offers several methods to infer spherical statistics as spherical variance, Fisher's statistics, confidence cones on data etc.

In [ ]:
g.var

In [ ]:
g.fisher_stats

In [ ]:
g.delta

To calculate orientation tensor of all features in group, we can use ``ortensor`` property.

In [ ]:
g.ortensor

## Ortensor class
``Ortensor`` class represents orientation tensor of set of planar or linear features. Eigenvalues and eigenvectors could be obtained by methods ``eigenvals`` and ``eigenvects``. Eigenvectors could be also represented by linear or planar features using properties ``eigenlins`` and ``eigenfols``. Several properties to describe orientation distribution is also impleneted, e.g. Woodcock's ``shape`` and ``strength`` or Vollmer's ``P``, ``G``, ``R`` and ``C`` indexes.

In [ ]:
ot = Ortensor(g)
ot.eigenvals

In [ ]:
ot.eigenvects.data

In [ ]:
ot.eigenlins.data

In [ ]:
ot.eigenfols.data

In [ ]:
StereoNet(g, ot);

In [ ]:
ot.strength, ot.shape

In [ ]:
ot.P, ot.G, ot.R

## StereoNet class
When ``StereoNet`` class instance is created with arguments, they are immidiatelly plotted. Most of the objects provided by **APSG** could be plotted.

Hovewer, the instance of `StereoNet` class could be used to visualize individual features according user needs using `StereoNet` methods like, `plane`, `line` or `pole`.

In [ ]:
s = StereoNet()
s.plane(Fol(150, 40))
s.pole(Fol(150, 40))
s.line(Lin(112, 30))
s.show()

A small circles (or cones) could be plotted as well using method `cone`.

In [ ]:
s = StereoNet()
g = Group.randn_lin(mean=Lin(40, 15))
s.line(g, 'k.')
s.cone(g.R, g.fisher_stats['a95'], 'r')  # confidence cone on resultant
s.cone(g.R, g.fisher_stats['csd'], 'g')  # confidence cone on 63% of data
s.show()

To make density contours plots, a ``contour`` and ``contourf`` methods are available

In [ ]:
s = StereoNet()
g = Group.randn_lin(mean=Lin(40, 20))
s.contourf(g, 8, legend=True, sigma=2)
s.line(g, 'g.')
s.show()

Except ``Group``, **APSG** provides ``PairSet`` and ``FaultSet`` classes to store ``Pair`` or ``Fault`` datasets. It can be inicialized by passing list of ``Pair`` or ``Fault`` objects as argument or use class methods ``from_array`` or ``from_csv``

In [ ]:
p = PairSet([Pair(120, 30, 165, 20),
             Pair(215, 60, 280,35),
             Pair(324, 70, 35, 40)])
p.misfit

In [ ]:
StereoNet(p);

``StereoNet`` has two special methods to visualize fault data. Method ``fault`` produce classical Angelier plot

In [ ]:
f = FaultSet([Fault(170, 60, 182, 59, -1),
              Fault(210, 55, 195, 53, -1),
              Fault(10, 60, 15, 59, -1),
              Fault(355, 48, 22, 45, -1)])
s = StereoNet()
s.fault(f)
s.line(f.p, label='P-axes')
s.line(f.t, label='T-axes')
s.plane(f.m, label='M-planes')
s.show()

`hoeppner` method produce Hoeppner diagram

In [ ]:
s = StereoNet()
s.hoeppner(f, label='Faults')
s.show()

Note that ``fault`` method is used, when data are passed directly to ``StereoNet`` instance

In [ ]:
f = Fault(120, 60, 110, 58, -1)
StereoNet(f, f.m, f.d, f.p, f.t);

## StereoGrid class
``StereoGrid`` class allows to visualize any scalar field on StereoNet. Internally it is used for plotting contour diagrams, but it exposes ``apply_func`` method to calculate scalar field by any user-defined function. Function must accept three element ``numpy.array`` as first argument passed from grid points of ``StereoGrid``.

Following example defines function to calculate resolved shear stress on plane from given stress tensor. ``StereoGrid`` is used to calculate this value over all directions and finally values are plotted by ``StereoNet``

In [ ]:
S = Stress([[-10, 2, -3], [2, -5, 1], [-3, 1, 2]])
d = StereoGrid()
d.apply_func(S.shear_stress)
s = StereoNet()
s.contourf(d, 10, legend=True)
s.show()

The ``FaultSet`` provide also ``amgmech`` method which provide access to Angelier dihedra method. Results are stored in ``StereoGrid``. Default behavior is to calculate counts (positive in extension, negative in compression)

In [ ]:
f = FaultSet.examples('MELE')
StereoNet(f);

In [ ]:
s = StereoNet()
s.contourf(f.angmech(), 6, legend=True)
s.show()

Setting method to 'probability', maximum likelihood estimate is calculated.

In [ ]:
s = StereoNet()
s.contourf(f.angmech(method='probability'), 6, legend=True)
s.show()

## FabricPlot class
``FabricPlot`` class provide triangular fabric plot (Vollmer, 1989). You can pass either ``Ortensor`` or ``Group`` instances

In [ ]:
g1 = Group.examples('B2')
g2 = Group.examples('B4')
g3 = Group.uniform_lin(name='Uniform')
FabricPlot(g1, g2, g3);

## Cluster class
``Cluster`` class provide access to **scipy** hierarchical clustering. Distance matrix is calculated as mutual angles of features within Group keeping axial and/or vectorial nature in mind. ``Cluster.explain`` method allows to explore explained variance versus number of clusters relation. Actual cluster is done by ``Cluster.cluster`` method, using distance or maxclust criterion. Using of ``Cluster`` is explained in following example. We generate some data and plot dendrogram

In [ ]:
g1 = Group.randn_lin(mean=Lin(45,30))
g2 = Group.randn_lin(mean=Lin(320,56))
g3 = Group.randn_lin(mean=Lin(150,40))
g = g1 + g2 + g3
cl = Cluster(g)
cl.dendrogram(no_labels=True)

Now we can explore evolution of within-groups variance versus number of clusters on Elbow plot (Note change in slope for three clusters)

In [ ]:
cl.elbow()

Finally we can do clustering and plot created clusters

In [ ]:
cl.cluster(maxclust=3)
cl.R.data  # Restored centres of clusters

In [ ]:
StereoNet(*cl.groups, cl.R);

## Some tricks

Double cross products are allowed but not easy to understand.

For example ``p**l**p`` is interpreted as ``p**(l**p)``: a) ``l**p`` is plane defined by ``l`` and ``p`` normal b) intersection of this plane and ``p`` is calculated

In [ ]:
p = Fol(250,40)
l = Lin(160,25)
s = StereoNet()
s.plane(p, lw=3, label='p')
s.line(l, ms=10, label='l')
s.plane(l**p, label='l**p')
s.line(p**l, label='p**l')
s.plane(l**p**l, label='l**p**l')
s.line(p**l**p, label='p**l**p')

``Pair`` class could be used to correct measurements of planar linear features which should spatialy overlap

In [ ]:
pl = Pair(250, 40, 160, 25)
pl.misfit

In [ ]:
s = StereoNet()
s.plane(Fol(250, 40), 'b', label='Original')
s.line(Lin(160, 25), 'bo', label='Original')
s.plane(pl.fol, 'g', label='Corrected')
s.line(pl.lin, 'go', label='Corrected')
s.show()

``StereoNet`` has method ``arrow`` to draw arrow. Here is example of Hoeppner plot for variable fault orientation within given stress field

In [ ]:
S = Stress([[-8, 0, 0],[0, -5, 0],[0, 0, -1]]).rotate(Lin(90,45), 45)
d = StereoGrid(npoints=300)
#d.apply_func(S.shear_stress)
s = StereoNet()
#s.contourf(d, 10, legend=True, alpha=0.3)
s.tensor(S)
for dc in d.dcgrid:
    f = S.fault(dc)
    s.arrow(f.fvec, f.lvec, f.sense)
s.show()

In [ ]:
d = StereoGrid()
d.apply_func(S.normal_stress)
s = StereoNet()
s.contourf(d, 10, legend=True)
s.show()

In [ ]:
d = StereoGrid()
d.apply_func(S.shear_stress)
s = StereoNet()
s.contourf(d, 10, legend=True)
s.show()